# Combining Our Local Data Sources

## Combining `.tif` File Data

In [3]:
from rasterio.plot import plotting_extent
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rasterio

In [6]:
# gsa

tif_files = {}
datasets = {}
for i in range (1, 2):
    if i < 10:
        tif_file_wst = f"../Data/gsa/westhem-monthly/PVOUT_0{i}.tif"
    else:
        tif_file_wst = f"../Data/gsa/westhem-monthly/PVOUT_{i}.tif"
    tif_files[i] = tif_file_wst
    
    # Reading the .tif file
    with rasterio.open(tif_file_wst) as ds:
        pvo_data = ds.read(1)  # Reading the first band
        
        # Data dimensions
        h = ds.height
        w = ds.width
        t = ds.transform

        # Data values
        lats = []
        longs = []
        pvo_vals = []

        # Extracting the information of each data points from processed .tif file
        for row in range(h):
            for col in range(w):
                pvo_val = pvo_data[row, col]
                if pvo_val != ds.nodata and not np.isnan(pvo_val):
                    
                    # Converting pixel coords to geographic coords
                    lat, long = ds.xy(t, row, col)
                    lats.append(lat)
                    longs.append(long)
                    pvo_vals.append(pvo_val)

        # Creating a DataFrame
        df = pd.DataFrame({
            'latitude': lats,
            'longitude': longs,
            'pvo': pvo_vals
        })
        datasets[i] = df            

print(datasets[1].head())

                                            latitude  \
0  [-178.97916666666666, -178.97916666666666, -17...   
1  [-178.97916666666666, -178.97916666666666, -17...   
2  [-178.97916666666666, -178.97916666666666, -17...   
3  [-178.97916666666666, -178.97916666666666, -17...   
4  [-178.97916666666666, -178.97916666666666, -17...   

                                           longitude    pvo  
0  [60.99576388888889, 60.99583333333333, 62.4958...  0.947  
1  [60.99576388888889, 60.99583333333333, 62.4958...  0.926  
2  [60.99576388888889, 60.99583333333333, 62.4958...  0.905  
3  [60.99576388888889, 60.99583333333333, 62.4958...  0.884  
4  [60.99576388888889, 60.99583333333333, 62.4958...  0.855  


In [37]:
# nrel

irradiance_file = "../Data/nrel/monthly-ghi/ghi_apr.tif"

with rasterio.open(irradiance_file) as ds:
    irr_data = ds.read(1) 
    
    height = ds.height
    width = ds.width
    transform = ds.transform
    
    lats, longs, irr_vals = [], [], []
    
    # Extract data points
    for row in range(height):
        for col in range(width):
            irr_val = irr_data[row, col]
            if irr_val != ds.nodata and not np.isnan(irr_val):
                # Convert pixel coords to geographic coords
                long, lat = rasterio.transform.xy(transform, row, col)
                lats.append(lat)
                longs.append(long)
                irr_vals.append(irr_val)
    
    irr_df = pd.DataFrame({
        'latitude': lats,
        'longitude': longs,
        'irradiance': irr_vals
    })

print(irr_df.head())
datasets[2] = irr_df

   latitude  longitude  irradiance
0     59.97    -167.58  138.034978
1     59.97    -167.54  138.196966
2     59.97    -167.50  139.161330
3     59.97    -167.46  140.819042
4     59.97    -167.42  139.861075


In [28]:
# gmaps

import googlemaps
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")
gmaps = googlemaps.Client(key=API_KEY)

latitudes = np.arange(-21, 62.25, 0.25)
longitudes = np.arange(-180, -21.25, 0.25)

grid_points = [(lat, lng) for lat in latitudes for lng in longitudes]

def batch_points(points, batch_size=512):
    for i in range(0, len(points), batch_size):
        yield points[i:i + batch_size]

import requests
import time 

BASE_URL = "https://maps.googleapis.com/maps/api/elevation/json"

def get_elevation_batch(batch):
    locations = "|".join(f"{lat},{lng}" for lat, lng in batch)
    params = {
        "locations": locations,
        "key": API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        if data["status"] == "OK":
            return data["results"]
        else:
            print("Error in response:", data["status"])
    else:
        print("HTTP Error:", response.status_code)
    return []

elevation_data = []

for batch in batch_points(grid_points):
    results = get_elevation_batch(batch)
    for res in results:
        elevation_data.append({
            "latitude": res["location"]["lat"],
            "longitude": res["location"]["lng"],
            "Elevation": res["elevation"]
        })
    time.sleep(0.2)  # respect API rate limits

# 5. Convert to DataFrame
df_gmaps = pd.DataFrame(elevation_data)
datasets[3] = df_gmaps
datasets[3].head()

Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE
Error in response: DATA_NOT_AVAILABLE


,latitude,longitude,Elevation
0,-21.0,-180.00,-3165.399902
1,-21.0,-179.75,-2379.611816
2,-21.0,-179.50,-1268.597046
3,-21.0,-179.25,-1585.173340
4,-21.0,-179.00,-1372.127441


In [29]:
for i in range(1, 4):
    print("latitude max: ", datasets[i]['latitude'].max())
    print("latitude min: ", datasets[i]['latitude'].min())
    print("longitude max: ", datasets[i]['longitude'].max())
    print("longitude min: ", datasets[i]['longitude'].min())
    print("\n")

latitude max:  -137.82083
latitude min:  -178.97917
longitude max:  60.99576
longitude min:  60.99576


latitude max:  59.97
latitude min:  -20.99
longitude max:  -22.5
longitude min:  -179.98


latitude max:  62.0
latitude min:  -21.0
longitude max:  -21.5
longitude min:  -180.0




In [40]:
# Combining

def round_to_quarter(x):
    return round(x * 4) / 4

for df in datasets.values():
    df['latitude'] = df['latitude'].apply(lambda x: x[0] if isinstance(x, (list, tuple, np.ndarray)) else x)
    df['longitude'] = df['longitude'].apply(lambda x: x[0] if isinstance(x, (list, tuple, np.ndarray)) else x)

    df['latitude'] = df['latitude'].astype(float).apply(round_to_quarter)
    df['longitude'] = df['longitude'].astype(float).apply(round_to_quarter)

# merged_df = datasets[1].merge(datasets[2], on=['latitude', 'longitude'])
# merged_df = merged_df.merge(datasets[3], on=['latitude', 'longitude'])
merged_df = datasets[2].merge(datasets[3], on=['latitude', 'longitude'])
merged_df.head()

,latitude,longitude,irradiance,Elevation
0,60.0,-168.0,138.034978,-42.154568
1,60.0,-168.0,138.034978,-37.598675
2,60.0,-168.0,138.034978,-34.660065
3,60.0,-168.0,138.034978,-37.685432
4,60.0,-168.0,138.034978,-25.348148


In [41]:
merged_df.shape

(31088923, 4)